In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

In [25]:
ds = tfds.load('diamonds', split='all')
assert isinstance(ds, tf.data.Dataset)
print(ds)

<_PrefetchDataset element_spec={'features': {'carat': TensorSpec(shape=(), dtype=tf.float32, name=None), 'clarity': TensorSpec(shape=(), dtype=tf.int64, name=None), 'color': TensorSpec(shape=(), dtype=tf.int64, name=None), 'cut': TensorSpec(shape=(), dtype=tf.int64, name=None), 'depth': TensorSpec(shape=(), dtype=tf.float32, name=None), 'table': TensorSpec(shape=(), dtype=tf.float32, name=None), 'x': TensorSpec(shape=(), dtype=tf.float32, name=None), 'y': TensorSpec(shape=(), dtype=tf.float32, name=None), 'z': TensorSpec(shape=(), dtype=tf.float32, name=None)}, 'price': TensorSpec(shape=(), dtype=tf.float32, name=None)}>


In [26]:
data = tfds.as_dataframe(ds)
diamonds = pd.DataFrame(data)
diamonds

,features/carat,features/clarity,features/color,features/cut,features/depth,features/table,features/x,features/y,features/z,price
0,1.26,2,4,2,60.599998,60.0,6.97,7.00,4.23,6546.0
1,0.80,3,4,4,62.099998,54.0,5.96,5.99,3.71,3030.0
2,0.56,4,2,4,61.700001,54.0,5.28,5.32,3.27,1915.0
3,1.51,3,6,1,64.000000,58.0,7.24,7.27,4.64,6936.0
4,0.33,6,5,4,62.200001,54.0,4.43,4.45,2.76,646.0
...,...,...,...,...,...,...,...,...,...,...
53935,1.02,2,4,2,62.900002,59.0,6.38,6.41,4.02,4796.0
53936,0.93,2,4,3,61.799999,59.0,6.27,6.23,3.86,3750.0
53937,0.30,4,5,4,62.400002,54.0,4.30,4.32,2.69,491.0
53938,0.36,3,2,4,61.500000,55.0,4.60,4.57,2.82,807.0


Izbrisacemo prefiks "*features/*" 

In [27]:
old_feature_names = diamonds.columns.to_list()
old_feature_names

['features/carat',
 'features/clarity',
 'features/color',
 'features/cut',
 'features/depth',
 'features/table',
 'features/x',
 'features/y',
 'features/z',
 'price']

In [28]:
def rename_features(features):

  features.remove('price')
  new_features = []
  for f in features:
    new_features.append(f[9:])
  new_features.append('price')

  return new_features


In [29]:
feature_names = rename_features(old_feature_names)
feature_names

['carat', 'clarity', 'color', 'cut', 'depth', 'table', 'x', 'y', 'z', 'price']

In [30]:
diamonds.columns = feature_names
diamonds.head()

,carat,clarity,color,cut,depth,table,x,y,z,price
0,1.26,2,4,2,60.599998,60.0,6.97,7.00,4.23,6546.0
1,0.80,3,4,4,62.099998,54.0,5.96,5.99,3.71,3030.0
2,0.56,4,2,4,61.700001,54.0,5.28,5.32,3.27,1915.0
3,1.51,3,6,1,64.000000,58.0,7.24,7.27,4.64,6936.0
4,0.33,6,5,4,62.200001,54.0,4.43,4.45,2.76,646.0


In [31]:
diamonds.describe()

,carat,clarity,color,cut,depth,table,x,y,z,price
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,3.051020,2.594197,2.904097,61.749405,57.457184,5.731157,5.734526,3.538734,3932.799805
std,0.474011,1.647136,1.701105,1.116600,1.432621,2.234491,1.121761,1.142135,0.705699,3989.439697
min,0.200000,0.000000,0.000000,0.000000,43.000000,43.000000,0.000000,0.000000,0.000000,326.000000
25%,0.400000,2.000000,1.000000,2.000000,61.000000,56.000000,4.710000,4.720000,2.910000,950.000000
50%,0.700000,3.000000,3.000000,3.000000,61.799999,57.000000,5.700000,5.710000,3.530000,2401.000000
75%,1.040000,4.000000,4.000000,4.000000,62.500000,59.000000,6.540000,6.540000,4.040000,5324.250000
max,5.010000,7.000000,6.000000,4.000000,79.000000,95.000000,10.740000,58.900002,31.799999,18823.000000


***

#### Nedostajuce vrednosti


In [32]:
diamonds.isna().any()

carat      False
clarity    False
color      False
cut        False
depth      False
table      False
x          False
y          False
z          False
price      False
dtype: bool

Kao sto vidimo, NaN vrednosti nema

***

#### Regression -> Classification

Posto regresioni problem svodimo na klasifikacioni, potrebno je kontinualni opseg atributa *price* podeliti na kategorije  
  
  Imacemo 4 kategorije po opsegu cena (u USD):

1. [0, 5000]
2. (5000, 10000]
3. (10000, 15000]
4. 15000+


In [33]:
priceClasses = ['[0, 5000]', '(5000, 10000]', '(10000, 15000]', '15000+']
classColumn = []
d = diamonds['price']
for i in d:
  if i >= 0 and i <= 5000:
    classColumn.append(priceClasses[0])
  elif i <= 10000:
    classColumn.append(priceClasses[1])
  elif i <= 15000:
    classColumn.append(priceClasses[2])
  elif i > 15000:
    classColumn.append(priceClasses[3])

In [34]:
s = pd.Series(classColumn)
diamonds['class'] = s
diamonds

,carat,clarity,color,cut,depth,table,x,y,z,price,class
0,1.26,2,4,2,60.599998,60.0,6.97,7.00,4.23,6546.0,"(5000, 10000]"
1,0.80,3,4,4,62.099998,54.0,5.96,5.99,3.71,3030.0,"[0, 5000]"
2,0.56,4,2,4,61.700001,54.0,5.28,5.32,3.27,1915.0,"[0, 5000]"
3,1.51,3,6,1,64.000000,58.0,7.24,7.27,4.64,6936.0,"(5000, 10000]"
4,0.33,6,5,4,62.200001,54.0,4.43,4.45,2.76,646.0,"[0, 5000]"
...,...,...,...,...,...,...,...,...,...,...,...
53935,1.02,2,4,2,62.900002,59.0,6.38,6.41,4.02,4796.0,"[0, 5000]"
53936,0.93,2,4,3,61.799999,59.0,6.27,6.23,3.86,3750.0,"[0, 5000]"
53937,0.30,4,5,4,62.400002,54.0,4.30,4.32,2.69,491.0,"[0, 5000]"
53938,0.36,3,2,4,61.500000,55.0,4.60,4.57,2.82,807.0,"[0, 5000]"


Kolona price nam vise nije od znacaja pa je mozemo odbaciti

In [35]:
diamonds.drop(['price'], axis='columns', inplace = True)
diamonds.head()

,carat,clarity,color,cut,depth,table,x,y,z,class
0,1.26,2,4,2,60.599998,60.0,6.97,7.00,4.23,"(5000, 10000]"
1,0.80,3,4,4,62.099998,54.0,5.96,5.99,3.71,"[0, 5000]"
2,0.56,4,2,4,61.700001,54.0,5.28,5.32,3.27,"[0, 5000]"
3,1.51,3,6,1,64.000000,58.0,7.24,7.27,4.64,"(5000, 10000]"
4,0.33,6,5,4,62.200001,54.0,4.43,4.45,2.76,"[0, 5000]"


***

#### Kodiranje kategorickih vrednosti

In [36]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float32
 1   clarity  53940 non-null  int64  
 2   color    53940 non-null  int64  
 3   cut      53940 non-null  int64  
 4   depth    53940 non-null  float32
 5   table    53940 non-null  float32
 6   x        53940 non-null  float32
 7   y        53940 non-null  float32
 8   z        53940 non-null  float32
 9   class    53940 non-null  object 
dtypes: float32(6), int64(3), object(1)
memory usage: 2.9+ MB


Mozemo uociti da su atributi *clarity*, *cut* i *color* vec celobrojno kodirani pa nema potrebe obradjivati ove kolone, ali moramo kodirati klasu koju zelimo da predvidimo  
Kodiranje cemo izvrsiti rucno, umesto da koristimo **OrdinalEncoder**, **OneHotEncoder** i sl., kako bismo kasnije mogli da iskoristimo PCA(Principal Component Analysis) za smanjenje dimenzije. (OE sortira prvo stringove pa ih kodira i dobijemo preslikavanje iz  [ '[0, 5000]', '(5000, 10000]', '(10000, 15000]', '15000+' ] u [ 1, 3, 0, 2 ] sto nam malo kvari matricu korelacije jer se ne vidi linearna povezanost atributa sa opsegom cene)

In [37]:
classDict = {'[0, 5000]' : 0,
             '(5000, 10000]' : 1,
             '(10000, 15000]' : 2,
             '15000+' : 3}

classColumn = []
d = diamonds['class']
for i in d:
  if i in classDict:
    classColumn.append(classDict[i])

In [38]:
diamonds['class'] = classColumn

In [39]:
diamonds

,carat,clarity,color,cut,depth,table,x,y,z,class
0,1.26,2,4,2,60.599998,60.0,6.97,7.00,4.23,1
1,0.80,3,4,4,62.099998,54.0,5.96,5.99,3.71,0
2,0.56,4,2,4,61.700001,54.0,5.28,5.32,3.27,0
3,1.51,3,6,1,64.000000,58.0,7.24,7.27,4.64,1
4,0.33,6,5,4,62.200001,54.0,4.43,4.45,2.76,0
...,...,...,...,...,...,...,...,...,...,...
53935,1.02,2,4,2,62.900002,59.0,6.38,6.41,4.02,0
53936,0.93,2,4,3,61.799999,59.0,6.27,6.23,3.86,0
53937,0.30,4,5,4,62.400002,54.0,4.30,4.32,2.69,0
53938,0.36,3,2,4,61.500000,55.0,4.60,4.57,2.82,0


***

#### Outlier-i